In [1]:
#%pip install git+https://github.com/autonlab/auton-survival.git

##losses

In [5]:
!python -V

Python 3.8.18


In [6]:
from auton_survival.models.dsm.utilities import get_optimizer, _reshape_tensor_with_nans

import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from copy import deepcopy

In [7]:
np.random.seed(982)
_ = torch.manual_seed(982)

In [8]:
def total_loss(model, x, t, e, alpha, beta, sigma):
    longitudinal_prediction, outcomes = model(x)
    t, e = t.long(), e.int()

    # Compute cumulative function from prediced outcomes
    cif = [torch.cumsum(ok, 1) for ok in outcomes]

    return (1 - alpha - beta) * longitudinal_loss(longitudinal_prediction, x) +\
              alpha * ranking_loss(cif, t, e, sigma) +\
              beta * negative_log_likelihood(outcomes, cif, t, e)

##utils

In [9]:
def train_ddh(model,
    x_train, t_train, e_train,
    x_valid, t_valid, e_valid,
    alpha, beta, sigma,
    n_iter = 80, lr = 1e-3,
    bs = 100, vbs = 500, cuda = True):

    optimizer = get_optimizer(model, lr)

    patience, old_loss = 0, np.inf
    nbatches = int(x_train.shape[0]/bs) + 1
    valbatches = int(x_valid.shape[0]/vbs) + 1

    for i in tqdm(range(n_iter)):
        model.train()
        for j in range(nbatches):
            xb = x_train[j*bs:(j+1)*bs]
            tb = t_train[j*bs:(j+1)*bs]
            eb = e_train[j*bs:(j+1)*bs]   
            if xb.shape[0] == 0:
                continue   
            if cuda:
                xb, tb, eb = xb.cuda(), tb.cuda(), eb.cuda()
  
            optimizer.zero_grad()
            loss = total_loss(model,xb,tb,eb, alpha, beta, sigma)
            # print('Loss value:', loss) #testing
            loss.backward()
            optimizer.step()
  
        model.eval()
        valid_loss = 0
        for j in range(valbatches):
            xb = x_valid[j*bs:(j+1)*bs]
            tb = t_valid[j*bs:(j+1)*bs]
            eb = e_valid[j*bs:(j+1)*bs]   
            if xb.shape[0] == 0:
                continue
  
            if cuda:
                xb, tb, eb = xb.cuda(), tb.cuda(), eb.cuda()
      
            valid_loss += total_loss(model, xb, tb, eb, alpha, beta, sigma)
  
        valid_loss = valid_loss.item()
        if valid_loss < old_loss:
                patience = 0
                old_loss = valid_loss
                best_param = deepcopy(model.state_dict())
        else:
            if patience == 5:
                break
            else:
                patience += 1
        print('Train Epoch: {} \tLoss: {:.6f}'.format(i, loss))
  
    model.load_state_dict(best_param)
    return model, best_param

In [10]:
def create_nn(inputdim, outputdim, dropout = 0.6, layers = [100, 100], activation = 'ReLU', no_activation_last = False):
    modules = []
    if dropout > 0:
        modules.append(nn.Dropout(p = dropout))    
    if activation == 'ReLU6':
        act = nn.ReLU6()
    elif activation == 'ReLU':
        act = nn.ReLU()
    elif activation == 'SeLU':
        act = nn.SELU()
    elif activation == 'Tanh':
        act = nn.Tanh()    
    prevdim = inputdim
    for hidden in layers + [outputdim]:
        modules.append(nn.Linear(prevdim, hidden, bias = True))
        modules.append(act)
        prevdim = hidden    
    if no_activation_last:
        modules = modules[:-1]    
    return nn.Sequential(*modules)

##ddh_torch

In [11]:
class DynamicDeepHitTorch(nn.Module):

    def __init__(self, input_dim, output_dim, layers_rnn,
                hidden_rnn, long_param = {}, att_param = {}, cs_param = {},
                typ = 'LSTM', optimizer = 'Adam', risks = 1):
        super(DynamicDeepHitTorch, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.optimizer = optimizer
        self.risks = risks
        self.typ = typ

        # RNN model for longitudinal data
        if self.typ == 'LSTM':
            self.embedding = nn.LSTM(input_dim, hidden_rnn, layers_rnn,
                                bias=False, batch_first=True)
        if self.typ == 'RNN':
            self.embedding = nn.RNN(input_dim, hidden_rnn, layers_rnn,
                                bias=False, batch_first=True,
                                nonlinearity='relu')
        if self.typ == 'GRU':
            self.embedding = nn.GRU(input_dim, hidden_rnn, layers_rnn,
                                bias=False, batch_first=True)

        # Longitudinal network
        self.longitudinal = create_nn(hidden_rnn, input_dim, no_activation_last = True, **long_param)

        # Attention mechanism
        self.attention = create_nn(input_dim + hidden_rnn, 1, no_activation_last = True, **att_param)
        self.attention_soft = nn.Softmax(1) # On temporal dimension

        # Cause specific network
        self.cause_specific = []
        for r in range(self.risks):
            self.cause_specific.append(create_nn(input_dim + hidden_rnn, output_dim, no_activation_last = True, **cs_param))
        self.cause_specific = nn.ModuleList(self.cause_specific)

        # Probability
        self.soft = nn.Softmax(dim = -1) # On all observed output

    def forward(self, x):
        """
            The forward function that is called when data is passed through DynamicDeepHit.
        """
        if x.is_cuda:
            device = x.get_device()
        else:
            device = torch.device("cpu")

        # RNN representation - Nan values for not observed data
        x = x.clone()
        inputmask = torch.isnan(x[:, :, 0])
        x[inputmask] = 0
        hidden, _ = self.embedding(x)        
        
#         print("input mask ",inputmask)
        
        # Longitudinal modelling
        longitudinal_prediction = self.longitudinal(hidden)

        # Attention using last observation to predict weight of all previously observed
        ## Extract last observation (the one used for predictions), index upto with data available for patient
        #example: each patient have 16 length i/p, so if he come for 8 times then last observation will be 7
        last_observations = ((~inputmask).sum(axis = 1) - 1)
#         print("last observation", last_observations)
        last_observations_idx = last_observations.unsqueeze(1).repeat(1, x.size(1)) #convert to linear of size 16 with index obtained above
        index = torch.arange(x.size(1)).repeat(x.size(0), 1).to(device)
#         print("last observation indez", last_observations_idx)
#         print("index ",index)

        last = index == last_observations_idx
#         print(last)
        x_last = x[last]

        ## Concatenate all previous with new to measure attention
        concatenation = torch.cat([hidden, x_last.unsqueeze(1).repeat(1, x.size(1), 1)], -1)

        ## Compute attention and normalize
        attention = self.attention(concatenation).squeeze(-1)
        attention[index >= last_observations_idx] = -1e10 # Want soft max to be zero as values not observed
        attention[last_observations > 0] = self.attention_soft(attention[last_observations > 0]) # Weight previous observation
        attention[last_observations == 0] = 0 # No context for only one observation

        # Risk networks
        # The original paper is not clear on how the last observation is
        # combined with the temporal sum, other code was concatenating them
        outcomes = []
        attention = attention.unsqueeze(2).repeat(1, 1, hidden.size(2))
        hidden_attentive = torch.sum(attention * hidden, axis = 1)
        hidden_attentive = torch.cat([hidden_attentive, x_last], 1)
        for cs_nn in self.cause_specific:
            outcomes.append(cs_nn(hidden_attentive))

        # Soft max for probability distribution
        outcomes = torch.cat(outcomes, dim = 1)
        outcomes = self.soft(outcomes)

        outcomes = [outcomes[:, i * self.output_dim : (i+1) * self.output_dim] for i in range(self.risks)]
        return longitudinal_prediction, outcomes

##ddh_api

In [12]:
k = torch.arange(4).repeat(2,5,1)
print(k)

tensor([[[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]],

        [[0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3],
         [0, 1, 2, 3]]])


In [13]:
from auton_survival.models.dsm import DeepRecurrentSurvivalMachines
from auton_survival.models.dsm.utilities import _reshape_tensor_with_nans, _get_padded_features, _get_padded_targets

In [10]:
last = lambda t: np.array([t_[-1] for t_ in t], dtype = float)

class DynamicDeepHit(DeepRecurrentSurvivalMachines):
    """
        This implementation considers that the last event happen at the same time for each patient
        The CIF is therefore simplified
        Args:
            DeepRecurrentSurvivalMachines
    """

    def __init__(self, split = 50, layers_rnn = 1, hidden_rnn = 10, typ = 'LSTM',
        long_param = {}, att_param = {}, cs_param = {},
        alpha = 0.1, beta = 0.1, sigma = 0.1, cuda = True):

        if isinstance(split, int):
            self.split = split
            self.split_time = None
        else:
            self.split = len(split) - 1
            self.split_time = split

        self.layers_rnn = layers_rnn
        self.hidden_rnn = hidden_rnn
        self.typ = typ
        
        self.long_param = long_param
        self.att_param = att_param
        self.cs_param = cs_param

        self.alpha = alpha
        self.beta = beta
        self.sigma = sigma

        self.cuda = cuda
        self.fitted = False

        self.model_state = None
          
    def _gen_torch_model(self, inputdim, optimizer, risks):
        model = DynamicDeepHitTorch(inputdim, self.split, self.layers_rnn, self.hidden_rnn,
                long_param = self.long_param, att_param = self.att_param, cs_param = self.cs_param,
                typ = self.typ, optimizer = optimizer, risks = risks).double()
        if self.cuda:
            model = model.cuda()
        return model

    def cpu(self):
        self.cuda = False
        if self.torch_model:
            self.torch_model = self.torch_model.cpu()
        return self

    def fit(self, x, t, e, vsize = 0.15, val_data = None,
          iters = 1, learning_rate = 1e-3, batch_size = 100,
          optimizer = "Adam", random_state = 100):
        discretized_t, self.split_time = self.discretize(t, self.split, self.split_time)
        processed_data = self._prepocess_training_data(x, last(discretized_t), last(e),
                                                    vsize, val_data,
                                                    random_state)
        x_train, t_train, e_train, x_val, t_val, e_val = processed_data
        inputdim = x_train.shape[-1]  #number of features
        
        
#         print("training size", x_train.shape)
#         print("validation size", x_val.shape)
#         print("test size", x.shape)

        maxrisk = int(np.nanmax(e_train.cpu().numpy()))

        model = self._gen_torch_model(inputdim, optimizer, risks=maxrisk)
        model, best_param  = train_ddh(model,
                    x_train, t_train.to(torch.device('cuda')), e_train.to(torch.device('cuda')),
                    x_val.to(torch.device('cuda')), t_val.to(torch.device('cuda')), e_val.to(torch.device('cuda')),
                    self.alpha, self.beta, self.sigma, 
                    n_iter = iters,
                    lr = learning_rate,
                    bs = batch_size,
                    cuda = self.cuda)

        self.torch_model = model.eval()
        self.fitted = True
        self.cpu()
    
        self.model_state = best_param

        return self  

    def discretize(self, t, split, split_time = None):
        """
            Discretize the survival horizon
            Args:
                t (List of Array): Time of events
                split (int): Number of bins
                split_time (List, optional): List of bins (must be same length than split). Defaults to None.
            Returns:
                List of Array: Disretized events time
        """
        if split_time is None:
            _, split_time = np.histogram(np.concatenate(t), split - 1)
        t_discretized = np.array([np.digitize(t_, split_time, right = True) - 1 for t_ in t], dtype = object)
        return t_discretized, split_time

    def _prepocess_test_data(self, x):
        data = torch.from_numpy(_get_padded_features(x))
        if self.cuda:
            data = data.cuda()
        return data

    def _prepocess_training_data(self, x, t, e, vsize, val_data, random_state):
        """RNNs require different preprocessing for variable length sequences"""

        idx = list(range(x.shape[0]))
        np.random.seed(random_state)
        np.random.shuffle(idx)

        x = _get_padded_features(x) #add nans to keep the dimension same for each patient
        x_train, t_train, e_train = x[idx], t[idx], e[idx]

        x_train = torch.from_numpy(x_train).double()
        t_train = torch.from_numpy(t_train).double()
        e_train = torch.from_numpy(e_train).double()
        #padded x_val is
#         print("padded x_val in model.fit",x_train)

        if val_data is None:

            vsize = int(vsize*x_train.shape[0])

            x_val, t_val, e_val = x_train[-vsize:], t_train[-vsize:], e_train[-vsize:]
            
            x_train = x_train[:-vsize]
            t_train = t_train[:-vsize]
            e_train = e_train[:-vsize]

        else:

            x_val, t_val, e_val = val_data

            x_val = _get_padded_features(x_val)
            t_val, _ = self.discretize(t_val, self.split, self.split_time)

            x_val = torch.from_numpy(x_val).double()
            t_val = torch.from_numpy(last(t_val)).double()
            e_val = torch.from_numpy(last(e_val)).double()
            

        return (x_train, t_train, e_train,                
                x_val, t_val, e_val)

    def compute_nll(self, x, t, e):
        if not self.fitted:
            raise Exception("The model has not been fitted yet. Please fit the " +
                                    "model using the `fit` method on some training data " +
                                    "before calling `_eval_nll`.")
        discretized_t, _ = self.discretize(t, self.split, self.split_time)
        processed_data = self._prepocess_training_data(x, last(discretized_t), last(e), 0, None, 0)
        _, _, _, x_val, t_val, e_val = processed_data
        return total_loss(self.torch_model, x_val, t_val, e_val, 0, 1, 1).item()

    def predict_survival(self, x, t, risk=1, all_step=False, bs=100):
        l = [len(x_) for x_ in x]

        if all_step:
            new_x = []
            for x_, l_ in zip(x, l):
                new_x += [x_[:li + 1] for li in range(l_)]
            x = new_x

        if not isinstance(t, list):
            t = [t]
        # print(self.discretize([t], self.split, self.split_time))
        t = self.discretize([t], self.split, self.split_time)[0][0]
        # print(t)
        if self.fitted:
            batches = int(len(x)/bs) + 1
            scores = {t_: [] for t_ in t}
            for j in range(batches):
                xb = self._prepocess_test_data(x[j*bs:(j+1)*bs])
                _, f = self.torch_model(xb)
                for t_ in t:
                    scores[t_].append(torch.cumsum(f[int(risk) - 1], dim = 1)[:, t_].unsqueeze(1).detach().numpy())
            return 1 - np.concatenate([np.concatenate(scores[t_], axis = 0) for t_ in t], axis = 1)
        else:
            raise Exception("The model has not been fitted yet. Please fit the " +
                            "model using the `fit` method on some training data " +
                            "before calling `predict_survival`.")

    def predict_risk(self, x, t, **args):
        if self.fitted:
            return 1-self.predict_survival(x, t, **args)
        else:
            raise Exception("The model has not been fitted yet. Please fit the " +
                            "model using the `fit` method on some training data " +
                            "before calling `predict_risk`.")
     

    def load_saved(self, x, t, e, best_param, vsize = 0.15, val_data = None, optimizer='Adam', random_state = 100):
        discretized_t, self.split_time = self.discretize(t, self.split, self.split_time)
        processed_data = self._prepocess_training_data(x, last(discretized_t), last(e),
                                                       vsize, val_data, random_state)
        x_train, t_train, e_train, x_val, t_val, e_val = processed_data
        inputdim = x_train.shape[-1]

        maxrisk = int(np.nanmax(e_train.cpu().numpy()))

        model = self._gen_torch_model(inputdim, optimizer, risks=maxrisk)
        model = self._gen_torch_model(inputdim, optimizer, risks=maxrisk)
        model.load_state_dict(best_param)
    
        self.torch_model = model.eval()
        self.fitted = True
        self.cpu()

        self.model_state = best_param

        return self  

#Example
Dynamic Deep Hit on PBC Dataset

In [11]:
from pandas.core.dtypes.cast import maybe_promote
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import torchvision

def load_pbc_dataset(sequential=True):
  """Helper function to load and preprocess the PBC dataset
  The Primary biliary cirrhosis (PBC) Dataset [1] is well known
  dataset for evaluating survival analysis models with time
  dependent covariates.
  Parameters
  ----------
  sequential: bool
    If True returns a list of np.arrays for each individual.
    else, returns collapsed results for each time step. To train
    recurrent neural models you would typically use True.
  References
  ----------
  [1] Fleming, Thomas R., and David P. Harrington. Counting processes and
  survival analysis. Vol. 169. John Wiley & Sons, 2011.
  """

  # data = pkgutil.get_data(__name__, 'datasets/pbc2.csv')
  data = pd.read_csv('pbc.csv')
  data = data.dropna(how='any')
  data['histologic'] = data['histologic'].astype(str)
  dat_cat = data[['drug', 'sex', 'ascites', 'hepatomegaly',
                  'spiders', 'edema', 'histologic']]
  dat_num = data[['serBilir', 'serChol', 'albumin', 'alkaline',
                  'SGOT', 'platelets', 'prothrombin']]
  age = data['age'] + data['years']

  x1 = pd.get_dummies(dat_cat).values
  x2 = dat_num.values
  x3 = age.values.reshape(-1, 1)
  x = np.hstack([x1, x2, x3])

  time = (data['years'] - data['year']).values
  event = data['status2'].values

  # x = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x) #why impute the values?
  x_ = StandardScaler().fit_transform(x)

  if not sequential:
    return x_, time, event
  else:
    x, t, e = [], [], []
    for id_ in sorted(list(set(data['id']))):
      x.append(x_[data['id'] == id_])
      t.append(time[data['id'] == id_])
      e.append(event[data['id'] == id_])
    return x, t, e


def load_real_dataset(sequential=True):
  data = pd.read_csv('')

  # x = data[['ALBUMIN, Serum', 'HAEMOGLOBIN', 'PARATHORMONE (intact), Plasma', 'PHOSPHORUS, Serum','POTASSIUM, Serum', 'Tobacco', 
  #           'Days of Hypertension prior dialysis', 'Hospitalization count', 'Access creation count', 'isDiabetic']].values
  x = data[['ALBUMIN, Serum', 'PARATHORMONE (intact), Plasma', 'PHOSPHORUS, Serum','POTASSIUM, Serum', 'Tobacco', 
            'Days of Hypertension prior dialysis', 'Hospitalization count', 'Access creation count', 'isDiabetic']].values

  time = data['time'].values
  event = data['Status'].values

  # x = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(x) #why impute the values?
  x_ = StandardScaler().fit_transform(x)
  x_ = np.nan_to_num(x_)

  if not sequential:
    return x_, time, event
  else:
    x, t, e = [], [], []
    for id_ in sorted(list(set(data['id']))):
      x.append(x_[data['id'] == id_])
      t.append(time[data['id'] == id_])
      e.append(event[data['id'] == id_])
    return x, t, e


In [12]:
x, t, e = load_pbc_dataset()
#x, t, e = load_real_dataset()

#t for time, e for event, x for data, for all patient
#every patient have different frequency

In [13]:
len(x)

304

In [14]:
#interested in death event time e[-1]==1
horizons = [0.25, 0.5, 0.75]
times = np.quantile([t_[-1] for t_, e_ in zip(t, e) if e_[-1] == 1], horizons).tolist()
#creating 4 intervals
split = [0] + times + [np.max([t_.max() for t_ in t])]
split

[0, 0.5996057386923662, 1.06368415288577, 2.234831891359113, 14.3056620304457]

In [15]:
n = len(x)

tr_size = int(n*0.70)
vl_size = int(n*0.10)
te_size = int(n*0.20)

x_train, x_test, x_val = np.array(x[:tr_size], dtype = object), np.array(x[-te_size:], dtype = object), np.array(x[tr_size:tr_size+vl_size], dtype = object)
t_train, t_test, t_val = np.array(t[:tr_size], dtype = object), np.array(t[-te_size:], dtype = object), np.array(t[tr_size:tr_size+vl_size], dtype = object)
e_train, e_test, e_val = np.array(e[:tr_size], dtype = object), np.array(e[-te_size:], dtype = object), np.array(e[tr_size:tr_size+vl_size], dtype = object) 

In [16]:
from sklearn.model_selection import ParameterGrid

layers = [[], [100], [100, 100], [100, 100, 100]]

param_grid = {
              'layers_rnn': [2, 3], 
              'hidden_long': layers,  #t represents the number of hidden units in each layer of the long-term memory layer.   
              'hidden_rnn': [50, 100], 
              'hidden_att': layers,  #layers passes to attention layer
              'hidden_cs': layers,   #layers passes to context sensitive layer
              'sigma': [0.1, 1, 3],
              'learning_rate' : [1e-3],
             }
params = ParameterGrid(param_grid) #make all combinations of above

#few of params listed below
#{'hidden_att': [100, 100, 100], 'hidden_cs': [100, 100, 100], 'hidden_long': [], 'hidden_rnn': 50, 'layers_rnn': 2, 'learning_rate': 0.001, 'sigma': 1}
#{'hidden_att': [100, 100, 100], 'hidden_cs': [100, 100, 100], 'hidden_long': [], 'hidden_rnn': 50, 'layers_rnn': 2, 'learning_rate': 0.001, 'sigma': 3}

In [17]:

def negative_log_likelihood(outcomes, cif, t, e):
    """
        Compute the log likelihood loss 
        This function is used to compute the survival loss
    """
    loss, censored_cif = 0, 0
    #outcomes is a list of length containing tensor of size [100,4]
#     print("outcomes tensor is ",outcomes)
    # print("size of outcomes is :", len(outcomes))
#     print("outcomes[0] tensor is ",outcomes[0])
    # print("size of tensor is ", outcomes[0].size())
    #outcome tensor give probability in 4 intervals, 100 neurons in last 
    
    for k, ok in enumerate(outcomes):
        # Censored cif
        censored_cif += cif[k][e == 0][:, t[e == 0]] #first select cifs corresponding to risk type and then select which are censored and further select

        # Uncensored
        selection = e == (k + 1)
        loss += torch.sum(torch.log(ok[selection][:, t[selection]] + 1e-10))

    # Censored loss
    loss += torch.sum(torch.log(1 - censored_cif + 1e-10))
    # print('nll:', - loss / len(outcomes))
    return - loss / len(outcomes)

def ranking_loss(cif, t, e, sigma):
    """
        Penalize wrong ordering of probability
        Equivalent to a C Index
        This function is used to penalize wrong ordering in the survival prediction
    """
    loss = 0
    # Data ordered by time
    for k, cifk in enumerate(cif):
        for ci, ti in zip(cifk[e-1 == k], t[e-1 == k]):
            # For all events: all patients that didn't experience event before
            # must have a lower risk for that cause
            if torch.sum(t > ti) > 0:
                # TODO: When data are sorted in time -> wan we make it even faster ?
                loss += torch.mean(torch.exp((cifk[t > ti][:, ti] - ci[ti])) / sigma)

    return loss / len(cif)

def longitudinal_loss(longitudinal_prediction, x):
    """
        Penalize error in the longitudinal predictions
        This function is used to compute the error made by the RNN
        NB: In the paper, they seem to use different losses for continuous and categorical
        But this was not reflected in the code associated (therefore we compute MSE for all)
        NB: Original paper mentions possibility of different alphas for each risk
        But take same for all (for ranking loss)
    """
    length = (~torch.isnan(x[:,:,0])).sum(axis = 1) - 1
    if x.is_cuda:
        device = x.get_device()
    else:
        device = torch.device("cpu")

    # Create a grid of the column index
    index = torch.arange(x.size(1)).repeat(x.size(0), 1).to(device)

    # Select all predictions until the last observed
    prediction_mask = index <= (length - 1).unsqueeze(1).repeat(1, x.size(1))

    # Select all observations that can be predicted
    observation_mask = index <= length.unsqueeze(1).repeat(1, x.size(1))
    observation_mask[:, 0] = False # Remove first observation

    return torch.nn.MSELoss(reduction = 'mean')(longitudinal_prediction[prediction_mask], x[observation_mask])

In [18]:
len(params)

768

In [19]:
%%time
models = []
param_map = {}
for param in params:
    model = DynamicDeepHit(
                layers_rnn = param['layers_rnn'],
                hidden_rnn = param['hidden_rnn'], 
                long_param = {'layers': param['hidden_long'], 'dropout': 0.1}, 
                att_param = {'layers': param['hidden_att'], 'dropout': 0.1}, 
                cs_param = {'layers': param['hidden_cs'], 'dropout': 0.1},
                sigma = param['sigma'],
                split = [0] + times + [np.max([t_.max() for t_ in t])])
    # The fit method is called to train the model
    model.fit(x_train, t_train, e_train, iters = 10, 
              learning_rate = param['learning_rate'])
    models.append([[model.compute_nll(x_val, t_val, e_val), model]])
    param_map[model] = param
best_model = min(models)
model = best_model[0][1]

torch.save(model.model_state, './orginal_code_best_model.pth')


 10%|████████▍                                                                           | 1/10 [00:04<00:42,  4.74s/it]

Train Epoch: 0 	Loss: 1256.380871


 20%|████████████████▊                                                                   | 2/10 [00:05<00:19,  2.48s/it]

Train Epoch: 1 	Loss: 1257.431371


 30%|█████████████████████████▏                                                          | 3/10 [00:06<00:11,  1.68s/it]

...

 60%|██████████████████████████████████████████████████▍                                 | 6/10 [00:00<00:00, 18.47it/s]

Train Epoch: 5 	Loss: 1204.697661
Train Epoch: 6 	Loss: 1203.890126


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.32it/s]

Train Epoch: 7 	Loss: 1202.744753
Train Epoch: 8 	Loss: 1201.461572
Train Epoch: 9 	Loss: 1198.916023


CPU times: user 53min 11s, sys: 51.6 s, total: 54min 2s
Wall time: 46min 1s


In [20]:
param = param_map[model]
param

{'hidden_att': [100, 100],
 'hidden_cs': [100, 100, 100],
 'hidden_long': [100],
 'hidden_rnn': 100,
 'layers_rnn': 2,
 'learning_rate': 0.001,
 'sigma': 0.1}

In [21]:
# #for original code
# param = {'hidden_att': [100, 100],
#  'hidden_cs': [100],
#  'hidden_long': [100, 100, 100],
#  'hidden_rnn': 100,
#  'layers_rnn': 2,
#  'learning_rate': 0.001,
#  'sigma': 1}

#for zero imputation
# param = {'hidden_att': [100],
#  'hidden_cs': [100, 100, 100],
#  'hidden_long': [],
#  'hidden_rnn': 50,
#  'layers_rnn': 3,
#  'learning_rate': 0.001,
#  'sigma': 0.1} 

param = {'hidden_att': [100,100],
 'hidden_cs': [100, 100, 100],
 'hidden_long': [100],
 'hidden_rnn': 100,
 'layers_rnn': 2,
 'learning_rate': 0.001,
 'sigma': 0.1} 

In [22]:
# saved_model = torch.load('/content/drive/MyDrive/MTP/orginal_code_best_model.pth')
saved_model = torch.load('./orginal_code_best_model.pth')
best_model = DynamicDeepHit(
            layers_rnn = param['layers_rnn'],
            hidden_rnn = param['hidden_rnn'], 
            long_param = {'layers': param['hidden_long'], 'dropout': 0.3}, 
            att_param = {'layers': param['hidden_att'], 'dropout': 0.3}, 
            cs_param = {'layers': param['hidden_cs'], 'dropout': 0.3},
            sigma = param['sigma'],
            split = [0] + times + [np.max([t_.max() for t_ in t])])

best_model.load_saved(x_train ,t_train,e_train,saved_model)
out_risk = best_model.predict_risk(x_test, times, all_step = True)
out_survival = best_model.predict_survival(x_test, times, all_step = True)

In [23]:
out_risk = model.predict_risk(x_test, times, all_step = True)
out_survival = model.predict_survival(x_test, times, all_step = True)

In [24]:
out_risk

array([[0.17313171, 0.37293354, 0.61233904],
       [0.16423867, 0.35858253, 0.59564011],
       [0.21761009, 0.42738478, 0.66526076],
       [0.21563672, 0.42663812, 0.66745243],
       [0.21836444, 0.42944621, 0.6728659 ],
       [0.22225999, 0.43478344, 0.67742168],
       [0.22190772, 0.43451452, 0.67732392],
       [0.2233525 , 0.43504245, 0.67255847],
       [0.22578646, 0.4395234 , 0.68097212],
       [0.14322014, 0.32566088, 0.54816321],
       [0.15306189, 0.34040037, 0.56767972],
       [0.13308865, 0.30916055, 0.52660094],
       [0.13243575, 0.30831348, 0.52508389],
       [0.14186666, 0.32656559, 0.54945013],
       [0.1391788 , 0.32031888, 0.54148712],
       [0.11358377, 0.27354749, 0.4754182 ],
       [0.09395808, 0.22893716, 0.40912141],
       [0.10828477, 0.26076478, 0.45449634],
       [0.1465624 , 0.33193629, 0.56142511],
       [0.15583222, 0.34886897, 0.58187715],
       [0.13460044, 0.31050979, 0.52822052],
       [0.07293173, 0.18606777, 0.34780227],
       [0.

In [25]:
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

cis = []
brs = []

et_train = np.array([(e_train[i][j], t_train[i][j]) for i in range(len(e_train)) for j in range(len(e_train[i]))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i][j], t_test[i][j]) for i in range(len(e_test)) for j in range(len(e_test[i]))],
                 dtype = [('e', bool), ('t', float)])
et_val = np.array([(e_val[i][j], t_val[i][j]) for i in range(len(e_val)) for j in range(len(e_val[i]))],
                 dtype = [('e', bool), ('t', float)])

for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
    brs.append(brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile,")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")

For 0.25 quantile,
TD Concordance Index: 0.9602467804396134
Brier Score: 0.027841668558642178
ROC AUC  0.9662969117017093 

For 0.5 quantile,
TD Concordance Index: 0.9675170972881088
Brier Score: 0.09198685812201805
ROC AUC  0.9801711160963481 

For 0.75 quantile,
TD Concordance Index: 0.8385437393827142
Brier Score: 0.2301983830440537
ROC AUC  0.8409912094823285 



In [30]:
times

[0.10061877121892449, 0.27242361187164976, 0.7077538057167874]

In [31]:
t_test[1]

array([6.6778009 , 6.11652612, 5.74143029])

In [32]:
model.predict_risk([x_test[1]], times, all_step = True)

array([[0.20238899, 0.43051931, 0.64449212],
       [0.16718445, 0.33829127, 0.53057285],
       [0.19727217, 0.42346649, 0.63642884]])

In [36]:
x_test[1]

array([[ 0.99436038, -0.99436038,  0.3725034 , -0.3725034 ,  0.36530793,
        -0.30847635, -0.97913787,  1.0425497 , -1.43799282,  1.54166667,
         0.62313247, -0.30446121, -0.49195313, -0.22660837, -0.39802969,
        -0.67758009,  1.00051427, -0.47891406, -0.58786896, -0.07931288,
        -0.38914062, -0.12076902,  0.86983368, -0.67486874,  1.73480403],
       [ 0.99436038, -0.99436038,  0.3725034 , -0.3725034 ,  0.36530793,
        -0.30847635, -0.97913787,  1.0425497 ,  0.69541377, -0.64864865,
         0.62313247, -0.30446121, -0.49195313, -0.22660837, -0.39802969,
        -0.67758009,  1.00051427, -0.47891406,  0.        ,  0.25872274,
        -0.24042292, -0.44212649,  0.97423179, -0.67486874,  1.73480403],
       [ 0.99436038, -0.99436038,  0.3725034 , -0.3725034 ,  0.36530793,
        -0.30847635, -0.97913787,  1.0425497 , -1.43799282,  1.54166667,
         0.62313247, -0.30446121, -0.49195313, -0.22660837, -0.39802969,
        -0.67758009,  1.00051427, -0.46029622, -0